In [1]:
import pandas as pd
import numpy as np

import glob
import tqdm
import gc

print('>>> 본 소스 작성 시 사용된 라이브러리 버전에 유의하세요!')
f'pd : {pd.__version__}  |  np : {np.__version__}  |  tqdm : {tqdm.__version__}'

>>> 본 소스 작성 시 사용된 라이브러리 버전에 유의하세요!


'pd : 1.0.5  |  np : 1.16.5  |  tqdm : 4.47.0'

## 셀-행정동 매핑자료 로드 (서울특별시 전체)

In [2]:
region_name_map = pd.read_csv('2_ADMDONG_CD-ADMDONG_NM.csv')
region_name_map = region_name_map[(region_name_map.sido_nm == '서울특별시')].reset_index(drop=True).drop(columns=['sido_cd', 'sido_nm'])

display(region_name_map.head())
print('서울특별시 소속 50셀 코드 수 :', region_name_map.shape[0])
print(f'서울특별시 소속 행정동 코드 (상위 5개) : {list(region_name_map.admdong_cd)[:5]} ...')

,sgg_cd,sgg_nm,admdong_cd,admdong_nm
0,11110,종로구,11110540,삼청동
1,11110,종로구,11110615,종로1.2.3.4가동
2,11110,종로구,11110650,혜화동
3,11110,종로구,11110680,창신2동
4,11110,종로구,11110700,숭인1동


서울특별시 소속 50셀 코드 수 : 424
서울특별시 소속 행정동 코드 (상위 5개) : [11110540, 11110615, 11110650, 11110680, 11110700] ...


In [3]:
cell_code_map = pd.read_csv('1_50CELL-ADMDONG_CD.csv').drop(columns=['x_axis', 'y_axis'])
cell_code_map = cell_code_map[cell_code_map.admdong_cd.isin(region_name_map.admdong_cd)]

display(cell_code_map.head())
print('서울특별시 소속 50셀 코드 수 :', cell_code_map.shape[0])
print(f'서울특별시 소속 50셀 코드 (상위 5개) : {list(cell_code_map.id)[:5]} ...')

,id,admdong_cd
757529,46047699,11500620
758464,46059888,11500620
758465,46059889,11500620
759402,46072077,11500620
759403,46072078,11500620


서울특별시 소속 50셀 코드 수 : 242363
서울특별시 소속 50셀 코드 (상위 5개) : [46047699, 46059888, 46059889, 46072077, 46072078] ...


## 행정동별 월별 유동인구 총합 계산 - 시간대별, 성연령별 (1월만)

In [4]:
# filename = 'SEOUL_201901*.csv'
# file_list = glob.glob(filename)
# year = int(filename[6:10])
# month = int(filename[10:12])

# time_list = []
# sexAge_list = []
# for file in file_list:
#     tmp_df = pd.read_csv(file,
#                          usecols=['id', 'timezn_cd', 'm00', 'm10', 'm15', 'm20', 'm25', 'm30',
#                                   'm35', 'm40', 'm45', 'm50', 'm55', 'm60', 'm65', 'm70', 'f00', 'f10',
#                                   'f15', 'f20', 'f25', 'f30', 'f35', 'f40', 'f45', 'f50', 'f55', 'f60',
#                                   'f65', 'f70', 'total'])
    
#     tmp_time_df = tmp_df[['id', 'timezn_cd', 'total']].merge(cell_code_map, how='left', on='id').\
#                     groupby(['admdong_cd', 'timezn_cd'])['total'].sum().reset_index()
#     tmp_sexAge_df = tmp_df.drop(columns=['timezn_cd', 'total']).merge(cell_code_map, how='left', on='id').\
#                         groupby('admdong_cd')[['m00', 'm10', 'm15', 'm20', 'm25', 'm30', 'm35', 'm40', 'm45', 'm50',
#                                                'm55', 'm60', 'm65', 'm70', 'f00', 'f10', 'f15', 'f20', 'f25', 'f30',
#                                                'f35', 'f40', 'f45', 'f50', 'f55', 'f60', 'f65', 'f70']].sum().reset_index()
#     tmp_sexAge_df = pd.melt(tmp_sexAge_df, id_vars=['admdong_cd'], var_name='sex_age', value_name='total')
    
#     del tmp_df
    
#     time_list.append(tmp_time_df)
#     sexAge_list.append(tmp_sexAge_df)
#     pass

# print(len(time_list), len(sexAge_list))

In [5]:
# # Generating monthly pop df by time
# time_month_df = time_list[0].merge(time_list[1], how='outer', on=['admdong_cd', 'timezn_cd']).\
#                             merge(time_list[2], how='outer', on=['admdong_cd', 'timezn_cd'])
# time_month_df['total'] = time_month_df[['total_x', 'total_y', 'total']].sum(axis=1)
# time_month_df['year'] = year
# time_month_df['month'] = month

# time_month_df = pd.merge(region_name_map,
#                          time_month_df[['admdong_cd', 'year', 'month', 'timezn_cd', 'total']],
#                          how='left', on='admdong_cd')
# time_month_df

In [6]:
# # Generating monthly pop df by sexAge
# sexAge_month_df = sexAge_list[0].merge(sexAge_list[1], how='outer', on=['admdong_cd', 'sex_age']).\
#                                 merge(sexAge_list[2], how='outer', on=['admdong_cd', 'sex_age'])
# sexAge_month_df['total'] = sexAge_month_df[['total_x', 'total_y', 'total']].sum(axis=1)
# sexAge_month_df['year'] = year
# sexAge_month_df['month'] = month

# sexAge_month_df = pd.merge(region_name_map,
#                            sexAge_month_df[['admdong_cd', 'year', 'month', 'sex_age', 'total']],
#                            how='left', on='admdong_cd')
# sexAge_month_df

## 행정동별 월별 유동인구 총합 계산 - 시간대별, 성연령별  (19년 1월-4월, 20년 1-4월)

In [7]:
time_result_list = []
sexAge_result_list = []

for year in [2019, 2020]:
    for month in tqdm.tqdm_notebook(range(1, 5)):
        month_str = f'0{month}'[-2:]
        filename = f'SEOUL_{year}{month_str}*.csv'
        file_list = glob.glob(filename)
        
        time_list = []
        sexAge_list = []
        for file in file_list:
            tmp_df = pd.read_csv(file,
                                 usecols=['id', 'timezn_cd', 'm00', 'm10', 'm15', 'm20', 'm25', 'm30',
                                          'm35', 'm40', 'm45', 'm50', 'm55', 'm60', 'm65', 'm70', 'f00', 'f10',
                                          'f15', 'f20', 'f25', 'f30', 'f35', 'f40', 'f45', 'f50', 'f55', 'f60',
                                          'f65', 'f70', 'total'])

            tmp_time_df = tmp_df[['id', 'timezn_cd', 'total']].merge(cell_code_map, how='left', on='id').\
                            groupby(['admdong_cd', 'timezn_cd'])['total'].sum().reset_index()
            tmp_sexAge_df = tmp_df.drop(columns=['timezn_cd', 'total']).merge(cell_code_map, how='left', on='id').\
                                groupby('admdong_cd')[['m00', 'm10', 'm15', 'm20', 'm25', 'm30', 'm35', 'm40', 'm45', 'm50',
                                                       'm55', 'm60', 'm65', 'm70', 'f00', 'f10', 'f15', 'f20', 'f25', 'f30',
                                                       'f35', 'f40', 'f45', 'f50', 'f55', 'f60', 'f65', 'f70']].sum().reset_index()
            tmp_sexAge_df = pd.melt(tmp_sexAge_df, id_vars=['admdong_cd'], var_name='sex_age', value_name='total')

            del tmp_df

            time_list.append(tmp_time_df)
            sexAge_list.append(tmp_sexAge_df)
            pass
        
        
        # Generating monthly pop df by time
        time_month_df = time_list[0].merge(time_list[1], how='outer', on=['admdong_cd', 'timezn_cd']).\
                                    merge(time_list[2], how='outer', on=['admdong_cd', 'timezn_cd'])
        time_month_df['total'] = time_month_df[['total_x', 'total_y', 'total']].sum(axis=1)
        time_month_df['year'] = year
        time_month_df['month'] = month
        
        time_month_df = pd.merge(region_name_map,
                                 time_month_df[['admdong_cd', 'year', 'month', 'timezn_cd', 'total']],
                                 how='left', on='admdong_cd')
        time_result_list.append(time_month_df)
        
        
        # Generating monthly pop df by sexAge
        sexAge_month_df = sexAge_list[0].merge(sexAge_list[1], how='outer', on=['admdong_cd', 'sex_age']).\
                                        merge(sexAge_list[2], how='outer', on=['admdong_cd', 'sex_age'])
        sexAge_month_df['total'] = sexAge_month_df[['total_x', 'total_y', 'total']].sum(axis=1)
        sexAge_month_df['year'] = year
        sexAge_month_df['month'] = month

        sexAge_month_df = pd.merge(region_name_map,
                                   sexAge_month_df[['admdong_cd', 'year', 'month', 'sex_age', 'total']],
                                   how='left', on='admdong_cd')
        sexAge_result_list.append(sexAge_month_df)
        
        
        gc.collect()

C:\Users\dslab01\anaconda3\envs\gis_prac\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


## 최종 데이터프레임 결측치 확인 및 저장

In [12]:
time_result_df = pd.concat(time_result_list, ignore_index=True)
display(time_result_df)
display(time_result_df.isna().sum())

time_result_df.to_pickle('서울특별시_월별_행정동별_시간대별_총합.pkl')

,sgg_cd,sgg_nm,admdong_cd,admdong_nm,year,month,timezn_cd,total
0,11110,종로구,11110540,삼청동,2019,1,0,41591
1,11110,종로구,11110540,삼청동,2019,1,1,27499
2,11110,종로구,11110540,삼청동,2019,1,2,19626
3,11110,종로구,11110540,삼청동,2019,1,3,16739
4,11110,종로구,11110540,삼청동,2019,1,4,18858
...,...,...,...,...,...,...,...,...
81403,11740,강동구,11740650,성내2동,2020,4,19,1028258
81404,11740,강동구,11740650,성내2동,2020,4,20,915544
81405,11740,강동구,11740650,성내2동,2020,4,21,800761
81406,11740,강동구,11740650,성내2동,2020,4,22,666749


sgg_cd        0
sgg_nm        0
admdong_cd    0
admdong_nm    0
year          0
month         0
timezn_cd     0
total         0
dtype: int64

In [13]:
sexAge_result_df = pd.concat(sexAge_result_list, ignore_index=True)
display(sexAge_result_df)
display(sexAge_result_df.isna().sum())

sexAge_result_df.to_pickle('서울특별시_월별_행정동별_성연령별_총합.pkl')

,sgg_cd,sgg_nm,admdong_cd,admdong_nm,year,month,sex_age,total
0,11110,종로구,11110540,삼청동,2019,1,m00,425
1,11110,종로구,11110540,삼청동,2019,1,m10,6414
2,11110,종로구,11110540,삼청동,2019,1,m15,58361
3,11110,종로구,11110540,삼청동,2019,1,m20,230662
4,11110,종로구,11110540,삼청동,2019,1,m25,386373
...,...,...,...,...,...,...,...,...
94971,11740,강동구,11740650,성내2동,2020,4,f50,689935
94972,11740,강동구,11740650,성내2동,2020,4,f55,766931
94973,11740,강동구,11740650,성내2동,2020,4,f60,635374
94974,11740,강동구,11740650,성내2동,2020,4,f65,335277


sgg_cd        0
sgg_nm        0
admdong_cd    0
admdong_nm    0
year          0
month         0
sex_age       0
total         0
dtype: int64